In [3]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from tabulate import tabulate
from matplotlib import pyplot as plt
import scipy.stats as st
import statsmodels.api as sm
import seaborn as sns
import functions as fun
import OLS_Estimation as OLS


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
df = pd.read_csv('Dataset.csv')
#df = df[df['Market share'] != 0]
Nobs=df['ID'].count()
df.head(5)
df['const']=np.ones((Nobs,1))
print(f'shape: {df.shape}')

shape: (1232, 14)


# Creating dummies

In [9]:
df2 = df.copy()
df2 = pd.get_dummies(df2, columns=['Segment'], drop_first=True)
df2 = pd.get_dummies(df2, columns=['Year'], drop_first=True)
df2['China'] = (df2['Country'] == 'CH').astype(int)

# Creating log market share

In [10]:
df2['Market share'] = df2['Market share'].replace(0, 1)
df2['log_market_share'] = np.log(df2['Market share'])
df2['log_market_share'].head(5)

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: log_market_share, dtype: float64

# Independent OLS

In [12]:
data = df2
print(f'shape: {data.shape}')
#data format
N = data['ID'].unique().size
T = df['Year'].unique().size
print(f'Data has N={N} and T={T} data is = {data.shape[0]}, {N*T}')
assert data.shape[0] == N*T, f'Error: data is not a balanced panel'



#Declare variables
y = data["log_market_share"].values.reshape((N*T,1))

variables = ['Price', 'HP', 'Chargetime', 'Range','Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J', 'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023', 'China']
xnames = ['Cons'] + variables

print(f'Variables: {len(variables)}')





# Estimate with OLS
#X = np.hstack([np.ones((N*T, 1))] + [data[var].values.reshape((N*T, 1)) for var in variables])
#results = fun.est_OLS(y, X, xnames)
# fun.logit(y, X, xnames)


shape: (1232, 31)
Data has N=112 and T=11 data is = 1232, 1232
Variables: 22


In [13]:
#X skal være [i:i+1,2:] 2: for at fjerne consten og pris
#beta skal være [2:] for at fjerne konstant og pris-koefficienten og reshapes for at kunne lave matrixmultiplikaiton. 

coefficients = OLS.get_coefficients_OLS(pd.read_csv('Dataset.csv'))
print(f'coefficients: {coefficients[:3]}, shape: {coefficients.shape}')

coefficients_labels = xnames[1:] #Fjerne konstanten

model_labels = data['Model'].unique()
j = 0
for i in model_labels:
    j = j
    #print(f'Car {j}: {i}')
    j = j+1  
print(f'models: {model_labels[:3]}')

alpha = coefficients[1]
beta = coefficients
p_j = data['Price'].values.reshape((N*T,1)) 

coefficients: const   -3.967207
Range    0.008889
Price   -0.000002
dtype: float64, shape: (23,)
models: ['U5' 'U6' 'e-tron']


In [14]:
def exp_delta(alpha, beta, X, p_j):
    s_j = []
    exp_s_j = []
    for i in range(len(p_j)):
        s = alpha*p_j[i] + X[i:i+1,2:]@beta[2:].reshape(-1,1)
        s_j.append(s)
        
    for i in range (len(s_j)):
        exp_s_j.append(np.exp(s_j[i]))
    return exp_s_j

def ccp(alpha, beta, X, p_j):
    ccp_list = [] 
    exp_delta_list = exp_delta(alpha, beta, X, p_j)
    sum_exp = np.sum(exp_delta_list)
    
    for i in range(len(exp_delta_list)):
        ccp_list.append(exp_delta_list[i]/sum_exp) 
    print(f'choice probability sum: {np.sum(ccp_list)} \n 3 highest probability: {np.sort(ccp_list, axis=0)[-3:]}')
    return ccp_list

ccp_list = ccp(alpha, beta, X, p_j)
print(f'ccp_list: {len(ccp_list)}')

NameError: name 'X' is not defined